# Source Files for Class Review

**Note:** The following files are provided for reference to follow along with the class discussion. You should not attempt to run the cells below, as the original training data is not for public distribution.

The training data the instructor will use locally is found in a directory called faces/, which you can see below. Note that there are 1,259 images we will use as our training data. The ground truth label for each image is embedded in it's file name. Files that begin with "f_" will be processed as female and files that begin with "m_" will be processed as male. Also, all the images have already been resized to the 120x100 size we will expect from the model we will build.

*These images originally come from public websites containing images of various countries members of congress.*

![My image folder to use as training data](images/faces_peek.png)

*What the "faces/" directory on my machine looks like.*

# 01 Train On Faces.py

In [ ]:
import random
import os
import numpy as np
import cv2

from keras.preprocessing.image import img_to_array

image_files_on_disk = []
data = []
labels = []

# STEP 1 - COLLECT ALL THE FILE NAMES IN THE 
# IMAGE FOLDER THAT HAS YOUR TRAINING DATA
for file in os.listdir("faces"):
    image_files_on_disk.append(file)

random.shuffle(image_files_on_disk)

# STEP 2 - FOR EACH FILE NAME, LOAD THE IMAGE,
# CONVERT IT TO AN ARRAY AND STORE IT IN OUR DATA ARRAY
# THE FILE NAME CONTAINS THE GENDER ASSIGNMENT WE WILL
# USE AS GROUND TRUTH. FILES BEGIN WITH EITHER "m_"
# OR "f_". IF THE FILE STARTS WITH "f_" WE WILL ASSIGN
# IT A LABEL OF 1 (female). OTHERWISE, WE WILL ASSUME 0 (male)
for file_name in image_files_on_disk:
    image = cv2.imread("faces/" + file_name)
    data.append( img_to_array(image))
    label = 0
    if file_name.startswith("f_"):
        label = 1
    labels.append([label])

# CONVERT OUR DATA AND LABELS TO NUMPY ARRAYS
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# ONE HOT ENCODE OUR LABELS
from keras.utils import to_categorical
labels = to_categorical(labels)

# STEP 3 - DEFINE OUR CNN MODEL
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import Adam

epochs = 100
lr = 1e-3
batch_size = 32

model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=(120, 100, 3)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation("sigmoid"))

# DEFINE OUR OPTIMIZER
opt = Adam(lr=lr, decay=lr/epochs)

# SINCE WE ONLY HAVE TWO CATEGORIES, WE COMPILE
# OUR MODEL AS A BINARY CLASSFIER
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# ONCE OUR MODEL IS COMPILED, WE TRAIN IT (fit)
# BY THE NUMBER OF EPOCHS WE DEFINED ABOVE
model.fit(data, labels,batch_size=batch_size, epochs=epochs, verbose=1 )

# ONCE WE ARE DONE, WE CAN SAVE THE MODEL AS FOLLOWS
model.save("face100.model")

# 02 Webcam test faces.py

In [ ]:
import cv2

from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np
import os

# STEP 1: LOAD THE MODEL FOR PREDICTION
model = load_model("face100.model")
# DEFINE HUMAN READABLE CLASS NAMES, SINCE THE
# MODEL PREDICTION WILL RETURN PROBABILITIES FOR
# 0 (male) AND 1 (female)
classes = ["man", "woman"]

# STEP 2: START WEBCAM
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("can not access webcam")
    exit()

# CONTINUOUSLY LOOP...
while webcam.isOpened():

    _, frame = webcam.read()
    # MY LOCAL WEBCAM IS 640x480 SO I'M ASSUMING
    # THE ABILITY TO CLIP A 400x480 BOX FROM THE MIDDLE
    # I REWRITE THIS TO MAKE IT USABLE FOR ANY WEBCAM LATER
    image = frame[0:640, 120:520]

    # NOW WE NEED TO GET OUR IMAGE READY TO BE
    # SUBMITTED TO OUR TRAINED MODEL FOR TESTING.
    # WE RESIZE, CONVERT TO FLOAT, AND CREATE 
    # A SINGLE IMAGE BATCH TO FEED INTO OUR MODEL
    tester = cv2.resize(image, (100, 120))
    tester = tester.astype("float") / 255.0
    tester = img_to_array(tester)
    tester = np.expand_dims(tester, axis=0)

    # WE NOW FEED IT TO OUR MODEL TO GET BACK OUR GENDER PREDICTIONS
    conf = model.predict(tester)[0]

    # WE CONVERT THE HIGHEST PREDICTION INTO A HUMAN READABLE LABEL
    idx = np.argmax(conf)
    label = classes[idx]

    # AND DRAW IT DIRECTLY INTO OUR WEBCAM IMAGE
    label = "{}: {:.2f}%".format(label, conf[idx] * 100)
    cv2.putText(image, label, (10, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    
    # WE THEN SHOW THE UPDATED WEBCAM IMAGE
    cv2.imshow("gender detection", image)

    # EXIT BY PRESSING 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

![Results from the first test](images/02_webcam_result.png)

*Not Good. We have no idea what the model learned!*

# 03 Preprocess data to boxedFaces.py

In [ ]:
import cv2
import os

# WE WILL USE OPENCV's FACE DETECTOR
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# FOR EACH FILE IN FACES, WE WANT TO LOAD INTO MEMORY
# THEN FEED A GREYSCALE COPY INTO OUR FACE DETECTOR.
for file_name in os.listdir("faces"):
    image = cv2.imread("faces/" + file_name)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(20,20) )

    # IF ANY FACES ARE FOUND, BLACK OUT THE TOP, BOTTOM AND SIDES
    # TO ONLY LEAVE THE FACE VISIBLE
    for ( x, y, w, h) in faces:
        cv2.rectangle(image, (0,0), (100,y), (0,0,0), -1)
        cv2.rectangle(image, (0,0), (x, 120), (0,0,0), -1)
        cv2.rectangle(image, (0, y+h), (100,120), (0,0,0), -1)
        cv2.rectangle(image, (x+w, 0), (100, 120), (0,0,0), -1)

    # SAVE THIS MODIFIED COPY INTO OUR NEW DIRECTORY
    newLocation = "boxedFaces/" + file_name
    cv2.imwrite(newLocation, image)
    

![My image folder to use as training data, now preprocessed to only show the face](images/boxedFaces_peek.png)

*What the "boxedFaces/" directory on my machine looks like. We are not attempting to only show the face*

# 04 Train On Boxed Faces.py

In [ ]:
import random
import os
import numpy as np
import cv2

from keras.preprocessing.image import img_to_array

image_files_on_disk = []
data = []
labels = []

'''
--------------------------------------------------------------
NOTE: THIS FILE IS ALMOST IDENTICAL TO "01 Train On Faces.py"
  THE ONLY DIFFERENCE IS THAT IT WILL LOOK AND READ FROM
  THE 'boxedFaces/' DIRECTORY INSTEAD OF THE ORIGINAL 
  'faces/' DIRECTORY. ALSO, IT SHOULD SAVE THE TRAINED
  model under the file name "boxedFace100.model"
  --------------------------------------------------------------
'''

# STEP 1 - COLLECT ALL THE FILE NAMES IN THE 
# IMAGE FOLDER THAT HAS YOUR TRAINING DATA
for file in os.listdir("boxedFaces"):
    image_files_on_disk.append(file)

random.shuffle(image_files_on_disk)

# STEP 2 - FOR EACH FILE NAME, LOAD THE IMAGE,
# CONVERT IT TO AN ARRAY AND STORE IT IN OUR DATA ARRAY
# THE FILE NAME CONTAINS THE GENDER ASSIGNMENT WE WILL
# USE AS GROUND TRUTH. FILES BEGIN WITH EITHER "m_"
# OR "f_". IF THE FILE STARTS WITH "f_" WE WILL ASSIGN
# IT A LABEL OF 1 (female). OTHERWISE, WE WILL ASSUME 0 (male)
for file_name in image_files_on_disk:
    image = cv2.imread("boxedFaces/" + file_name)
    data.append( img_to_array(image))
    label = 0
    if file_name.startswith("f_"):
        label = 1
    labels.append([label])

# CONVERT OUR DATA AND LABELS TO NUMPY ARRAYS
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# ONE HOT ENCODE OUR LABELS
from keras.utils import to_categorical
labels = to_categorical(labels)

# STEP 3 - DEFINE OUR CNN MODEL
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import Adam

epochs = 100
lr = 1e-3
batch_size = 32

model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=(120, 100, 3)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation("sigmoid"))

# DEFINE AN OPTIMIZER
opt = Adam(lr=lr, decay=lr/epochs)

# SINCE WE ONLY HAVE TWO CATEGORIES, WE COMPILE
# OUR MODEL AS A BINARY CLASSIFIER
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# ONCE OUR MODEL IS COMPILED, WE TRAIN IT (fit)
# BY THE NUMBER OF EPOCHS WE DEFINED ABOVE
model.fit(data, labels,batch_size=batch_size, epochs=epochs, verbose=1 )

# ONCE WE ARE DONE, WE CAN SAVE THE MODEL AS FOLLOWS:
model.save("boxedFace100.model")

![Results from the second test](images/04_webcam_result.png)

*It's Getting Better, but it is very jittery. Also, there is significant failure on female subjects. We need more training data...*

# 05 Train on Boxed Faces AUGMENTED.py

In [ ]:
import random
import os
import numpy as np
import cv2

from keras.preprocessing.image import img_to_array

image_files_on_disk = []
data = []
labels = []

'''
--------------------------------------------------------------
NOTE: THIS FILE IS VERY SIMILAR TO "04 Train On Boxed Faces.py"
  HOWEVER, NOTICE THAT WE ARE USING AN ImageDataGenerator TO
  AUGMENT EACH IMAGE FILE IN VARIOUS WAYS (scale, zoom, shearing, etc.)

  THIS MEANS THAT WE WILL ALSO CHANGE THE MODEL TRAINING COMMAND
  FROM 'model.fit()' TO 'model.fit_generator()' (see below)
  --------------------------------------------------------------
'''

# STEP 1 - COLLECT ALL THE FILE NAMES IN THE 
# IMAGE FOLDER THAT HAS YOUR TRAINING DATA
for file in os.listdir("boxedFaces"):
    image_files_on_disk.append(file)

# STEP 2 - FOR EACH FILE NAME, LOAD THE IMAGE,
# CONVERT IT TO AN ARRAY AND STORE IT IN OUR DATA ARRAY
# THE FILE NAME CONTAINS THE GENDER ASSIGNMENT WE WILL
# USE AS GROUND TRUTH. FILES BEGIN WITH EITHER "m_"
# OR "f_". IF THE FILE STARTS WITH "f_" WE WILL ASSIGN
# IT A LABEL OF 1 (female). OTHERWISE, WE WILL ASSUME 0 (male)
random.shuffle(image_files_on_disk)

for file_name in image_files_on_disk:
    image = cv2.imread("boxedFaces/" + file_name)
    data.append( img_to_array(image))
    label = 0
    if file_name.startswith("f_"):
        label = 1
    labels.append([label])

# CONVERT OUR DATA AND LABELS TO NUMPY ARRAYS
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

'''
# --------------------------------------------------
# **NEW**
# --------------------------------------------------
# WE ARE SPLITTING UP OUR DATA TO CREATE A SMALL AMOUNT
# OF TESTING DATA, THIS HELPS WITH IMPROVING ACCURACY
# NOTICE THAT data GETS SPLIT INTO 'trainX' and 'testX'
# AND THAT labels GETS SPLIT INTO 'trainY' and 'testY'
'''
from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.1, random_state=42)

# WHICH MEANS WE NOW NEED TO CONVERT BOTH OUR 
# TESTING AND TRAINING LABELS (trainY, testY) INTO
# ONE HOT ENCODED TOO...
from keras.utils import to_categorical
trainY = to_categorical(trainY, num_classes = 2)
testY = to_categorical(testY, num_classes = 2)

'''
# --------------------------------------------------
# **NEW**
# --------------------------------------------------
# AND HERE IS WHERE WE CREATE THE 'ImageDataGenerator' WHICH
# DEFINES HOW WE WANT TO AUGMENT EACH IMAGE DATA (rotation, shifting, shearing, zoom, scale, etc. )
# THIS IS A KEY WAY TO TURN A LITTLE BIT OF TRAINING DATA 
# INTO A LARGER AMOUNT, AS EACH ORIGINAL PICTURE CAN PRODUCE
# SEVERAL VARIATIONS, ALL SLIGHTLY DIFFERENT FROM EACH OTHER
'''
from keras.preprocessing.image import ImageDataGenerator
augmentedData = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.3, horizontal_flip=True, fill_mode="nearest")


# STEP 3 - DEFINE OUR CNN MODEL (no change)
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import Adam

epochs = 100
lr = 1e-3
batch_size = 32

model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=(120, 100, 3)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation("sigmoid"))

# DEFINE OUR OPTIMIZER
opt = Adam(lr=lr, decay=lr/epochs)

# SINCE WE ONLY HAVE TWO CATEGORIES, WE COMPILE
# OUR MODEL AS A BINARY CLASSIFIER
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


'''
# --------------------------------------------------
# **NEW**
# --------------------------------------------------
# WE ARE NOT USING JUST model.fit() AS WE DID BEFORE
# BUT WE ARE HAVING THE MODEL 'fit' TO ALL THE IMAGE
# VARIATIONS THAT WILL BE CREATED BY OUR ImageDataGenerator
# WHICH WE CALLED 'augmentedData'. WE ARE ALSO POINTING
# IT TO THE SMALL COLLECTION OF TESTING DATA SO WE CAN
# HAVE A MORE ACCURATE VALIDATION SCORE.
'''
train_session = model.fit_generator(augmentedData.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX, testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs,
                        verbose=1)

# ONE WE ARE DONE, WE CAN SAVE THE MODEL AS FOLLOWS:
model.save("boxedFace100augmented.model")

# 06 Webcam Test Boxed Faces (or augmented Boxed Faces).py

In [ ]:
import cv2

from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np
import os

# STEP 1: CHOOSE THE BOXED MODEL YOU WANT TO TEST
#model = load_model("boxedFace100.model")
model = load_model("boxedFace100augmented.model")

# DEFINE HUMAN READABLE CLASS NAMES, SINCE THE
# MODEL PREDICTION WILL RETURN PROBABILITIES FOR
# 0 (male) AND 1 (female)
classes = ["man", "woman"]

# STEP 2: LOAD OUR FACE DETECTOR
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# STEP 3: START WEBCAM
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("can not access webcam")
    exit()

# CONTINUOUSLY LOOP...
while webcam.isOpened():

    _, frame = webcam.read()

    # OUR MODEL IS EXPECTING A 100x120 IMAGE DATA ARRAY
    # SO WE NEED TO CLIP THE CAMERA INPUT (usually 640x480
    # BUT WILL DIFFER BY CAMERA) INTO A SLICE THAT SUPPORTS
    # A 5/6 ASPECT RATIO (i.e. 100/120). HERE'S HOW WE DO IT:
    (h, w, _) = frame.shape
    new_width = int(h*.833333)
    startX = int((w-new_width)/2)
    imgClipped = frame[:, startX:startX+new_width]

    gray = cv2.cvtColor(imgClipped, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(100,100) )

    for ( x, y, fw, fh) in faces:
        cv2.rectangle(imgClipped, (0,0), (w,y), (0,0,0), -1)
        cv2.rectangle(imgClipped, (0,0), (x, h), (0,0,0), -1)
        cv2.rectangle(imgClipped, (0, y+fh), (w,h), (0,0,0), -1)
        cv2.rectangle(imgClipped, (x+fw, 0), (w, h), (0,0,0), -1)

    imgTest = cv2.resize(imgClipped, (100, 120))
    imgTest = imgTest.astype("float") / 255.0
    imgTest = img_to_array(imgTest)
    imgTest = np.expand_dims(imgTest, axis=0)

    conf = model.predict(imgTest)[0]

    idx = np.argmax(conf)
    label = classes[idx]

    label = "{}: {:.2f}%".format(label, conf[idx] * 100)
    cv2.putText(imgClipped, label, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    cv2.imshow("gender detection", imgClipped)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

![Results from the second test](images/06_webcam_result.png)

*This one is better still, and there are a lot less false positives for males then before. Predictions are more stable too*